In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (col, monotonically_increasing_id)
from notebookUtils import readCsvAsDataframe

In [18]:
spark = SparkSession.builder.appName("factsales2").getOrCreate()

In [19]:
fact_sales = readCsvAsDataframe(spark,"fact_sales_data_v2.csv")

### Create Product Dimension

In [20]:
df_product = fact_sales.select([
    col('ProductName').alias('Title'),
    col('ProductCategory').alias('Category'),
    col('Brand')
]).distinct()

In [21]:
df_product_clean = df_product.dropna()

In [23]:
df_product2 = df_product_clean.withColumn("ProductKey",monotonically_increasing_id())

In [24]:
df_product_na = spark.createDataFrame([
    ("N/A","N/A","N/A",-1)
],["Name", "Category","Brand","ProductKey"])

In [25]:
dim_product = df_product2.unionAll(df_product_na)

In [26]:
dim_product.write.format("parquet").mode("overwrite").saveAsTable(name="dim_product")

In [27]:
df_dim_product = spark.read.parquet("spark-warehouse/dim_product")

In [28]:
df_dim_product.show(30)

[Row(Title='Tablet', Category='Clothing', Brand='BrandC', ProductKey=0),
 Row(Title='Smartphone', Category='Electronics', Brand='BrandB', ProductKey=1),
 Row(Title='Tablet', Category='Furniture', Brand='BrandA', ProductKey=2),
 Row(Title='Smartphone', Category='Furniture', Brand='BrandC', ProductKey=3),
 Row(Title='Desk', Category='Clothing', Brand='BrandB', ProductKey=4),
 Row(Title='Desk', Category='Furniture', Brand='BrandB', ProductKey=5),
 Row(Title='Desk', Category='Furniture', Brand='BrandA', ProductKey=6),
 Row(Title='Tablet', Category='Clothing', Brand='BrandA', ProductKey=7),
 Row(Title='Tablet', Category='Electronics', Brand='BrandB', ProductKey=8),
 Row(Title='Jeans', Category='Clothing', Brand='BrandC', ProductKey=9),
 Row(Title='Desk', Category='Clothing', Brand='BrandC', ProductKey=10),
 Row(Title='Tablet', Category='Furniture', Brand='BrandC', ProductKey=11),
 Row(Title='T-shirt', Category='Furniture', Brand='BrandB', ProductKey=12),
 Row(Title='Jeans', Category='Electr